# Bot Scrapper

This is a bot scrapper that downloads images from google images and saves them in a folder.


In [ ]:
search_string = '"mine field" -urban -city'
folder_path = "./dataSet/MineField/"

<span style="color:red">Don't change the code below.</span>


In [ ]:
%pip install selenium urllib3

In [ ]:
from __future__ import annotations  # noqa: F404

import os
import ssl
import time

import urllib3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

iter = 0

In [ ]:
def create_nested_folder(folder_path: str) -> None:
    folder_path = folder_path.rstrip("/").rstrip("\\")
    folders = folder_path.split(os.sep)

    path = "."
    for folder in folders:
        path = os.path.join(path, folder)
        if not os.path.exists(path):
            os.mkdir(path)

In [ ]:
def download_image(image_element) -> None:
    global iter
    global folder_path
    try:
        create_nested_folder(folder_path)

        http = urllib3.PoolManager(cert_reqs="CERT_NONE")
        response = http.request(
            "GET", image_element[0].get_attribute("src"), preload_content=False
        )
        with open(f"{folder_path}/{iter}.jpg", "wb") as out_file:
            out_file.write(response.data)
        response.release_conn()
        iter += 1
    except Exception as e:
        print(e)
        pass

In [ ]:
class Driver:
    def __init__(self: Driver) -> None:
        options = webdriver.ChromeOptions()
        options.add_argument("--remote-debugging-port=8000")
        self.driver = webdriver.Chrome()  # options.add_argument("--headless")
        self.wait = WebDriverWait(self.driver, 10)
        # self.driver.maximize_window()
        self.driver.set_window_size(1920, 1080)
        self.driver.get("https://www.google.com/")

    def setup(self: Driver) -> None:
        self.cookie_validator()

    def cookie_validator(self: Driver) -> None:
        self.wait.until(
            ec.presence_of_element_located((By.CSS_SELECTOR, "#L2AGLb > div")),
        )
        Driver.driver.find_element(By.CSS_SELECTOR, "#L2AGLb > div").click()

    def search(self: Driver, search_string: str) -> None:
        self.driver.get(
            f"https://www.google.com/search?q={search_string}&tbm=isch",
        )

    def get_images(self: Driver) -> list:
        return self.driver.find_elements(
            By.XPATH,
            '//*[@id="islrg"]/div[1]/div/a[1]/div/img',
        )

In [ ]:
class Item:
    def __init__(self: Item, url: str, driver: Driver) -> None:
        self.url = url
        self.Driver = driver

    def get_children(self: Item) -> list:
        return Driver.driver.find_elements(
            By.XPATH,
            '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/c-wiz/div/div/div/div[1]/div[3]/div/div/a[1]/div[1]/img',
        )

    def download(self, image_element: list) -> None:
        image_element[0].click()
        global folder_path
        global iter
        try:
            create_nested_folder(folder_path)

            http = urllib3.PoolManager(cert_reqs="CERT_NONE")
            response = http.request(
                "GET",
                image_element[0].get_attribute("src"),
                preload_content=False,
            )
            with open(f"{folder_path}/{iter}-fille.jpg", "wb") as out_file:
                out_file.write(response.data)
            response.release_conn()
            iter += 1
        except Exception as e:
            print(e)
            pass

        self.driver.execute_script("window.history.go(-1)")

In [ ]:
Driver = Driver()
Driver.setup()
Driver.search(search_string)
images = Driver.get_images()

In [ ]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


def find_elements(xpath):
    return WebDriverWait(Driver.driver, 10).until(
        ec.presence_of_all_elements_located((By.XPATH, xpath))
    )


for index, image in enumerate(images):
    try:
        image.click()

        main_image_xpath = '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'
        main_image_element = find_elements(main_image_xpath)
        print(main_image_element[0].get_attribute("src"))
        download_image(main_image_element)
        children_xpath = '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/c-wiz/div/div/div/div[1]/div[3]/div/div/a[1]/div[1]/img'
        children = find_elements(children_xpath)
        print(f"Found {len(children)} child images")
        time.sleep(1)

        for child_id, child in enumerate(children):
            try:
                children = find_elements(children_xpath)
                child = children[child_id]
                child.click()
                print(f"Found {len(children)} child images")
                time.sleep(1)
                child_image_xpath = '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'
                child_image_element = find_elements(main_image_xpath)
                print(child_image_element[0].get_attribute("src"))
                download_image(child_image_element)
                Driver.driver.execute_script("window.history.go(-1)")
            except Exception as e:
                print(f"Error processing child image {child_id}: {e}")
    except Exception as e:
        print(f"Error processing main image {index}: {e}")